This case would take an approach more like the Tassel paper, but in this case we do not pick a job as an action to take. We need to make an evaluation first of how to group them.
<ul>
  <li>Conditions
    <ul>
      <li>Use a binary array (as string)</li>
      <li>Lets say I pick the first 20 Jobs</li>
      <li>Then i have to get the the Recipes and quantities involved on each job</li>
      <li>My action set will be composed of Tuples (Recipe, Quantity, Parent_Job) where the Recipe is what i matters</li>
      <li>101000000000001010000 would be the action of which Recipes to execute</li>
      <li>The assignment should be automatic since no all machines would be able to take that</li>  
    </ul>
  </li>
</ul>


What is the maximum amount of recipes a Job can have. If i take max 20 jobs, then 20*max_amount_recipes_in_job is what i would need to define the action space.

For simplification matters I would say max_amount_recipes_in_job=3<br>
If we take max 15 jobs at a time, then<br>
ActionSpace = 15*3 = 45 -> 000000000000000000000000000000000000000000000


Action = 000000000000000000000000000000000000010000100 -> Do recipes 3 and 8

Having a string this way, means actions can also be represented by numbers and the binary represeation of the number will let us know which recipes are to be processed, for example, in the above case: <br>
Action = 000000000000000000000000000000000000010000100 -> Do recipes 3 and 8 = 2^2 + 2^7 = 4+128 = 132 
<br>
long datatype can fit up to 64 bits, so this approach can be extended up to 21 jobs with up to 3  recipes each.

In [1]:
class Machine:
    machine_type = "A" #A,B,C,D...
    knownRecipes = [] #R1,R2,R3... 
    trayCapacity = 10000
    status = 0    # 0-Free, 1-Busy - I am asuming you cannot open a machine that is workingt to add more trays
    activeRecipe = ""
    activeJobs = []
    startTime = ""
    
    def __init__(self, m_type, k_recipes, cap):
        self.machine_type = m_type
        self.knownRecipes=k_recipes
        self.trayCapacity = cap
        self.status = 0 #Free by default
        self.activeJobs= []
    
    def getKnownRecipes(self):
        return self.knownRecipes
    
    def getMachineType(self):
        return self.machine_type
    
    def getTrayCapacity(self):
        return self.trayCapacity
    
    def getStatus(self):
        return self.status
    
    def getTextStatus(self):
        if self.status == 0:
            return "Free"
        else:
            return "Busy"
    
    def updateKnownRecipes(self,new_recipes):
        self.knownRecipes=new_recipes
    
    def updateTrayCapacity(self,new_capacity):
        self.trayCapacity=new_capacity
    
    def assignJobs(self, jobs,recipe):
        assigned = self.setActiveRecipe(recipe)
        if assigned:
            for j in jobs:
                self.activeJobs.append(j)
                print(j)
                print(recipe)
                j.recipeInProgress(recipe)
            self.status = 1
            self.startTime = "now"
            return True
        else:
            return False
    
    def setActiveRecipe(self, recipe):
        if recipe in self.knownRecipes:
            self.activeRecipe = recipe
            return True
        else:
            return False
    
    def getActiveRecipe(self):
        return self.activeRecipe
    
    def getActiveJobsId(self):
        jobIds = ""
        for j in self.activeJobs:
            jobIds = jobIds + j.jobId + ", "
        return jobIds
    
    def resetMachine(self):
        self.status = 0    # 0-Free, 1-Busy - I am asuming you cannot open a machine that is workingt to add more trays
        self.activeRecipe = []
        self.activeJobs = []
    
    
    def __str__(self):
        return f'Type: {self.machine_type} \nKnown Recipes: {self.knownRecipes} \nTray Capacity: {self.trayCapacity} \nStatus: {self.getTextStatus()}\nWorking on recipe: {self.activeRecipe} for the following Jobs: {self.getActiveJobsId()}'
    


In [18]:
class Job:
    ''' This part is just to define the types. Not sure if it necessary'''
    jobId = "ID-00000" #String for now, can be numeric
    recipes = [] #R1,R2,R3...
    quantity = 0
    deadline = "3000/01/01" #YYYY/MM/DD
    status = 0
    priority = 1
    r_pending =[]
    r_inProgress = []
    r_completed=[]
    
    def __init__(self, jobId, recipes, quantity,deadline, priority):
        self.r_inProgress = []
        self.r_completed=[]
        self.r_pending=[]
        self.jobId = jobId
        self.recipes=recipes
        self.r_pending = recipes.copy()
        self.quantity = quantity
        self.deadline = deadline
        self.priority = priority # 1-Normal, 2-Medium, 3-High
        self.status = 0   # 0-New, 1- In Progress, 2- Completed, 3- Cancelled?, 4-Error? (For internal use)

    
    def getRecipes(self):
        return self.recipes
    
    def getJobId(self):
        return self.jobId
    
    def getQuantity(self):
        return self.quantity
    
    def getDeadline(self):
        return self.deadline
    
    def getStatus(self):
        return self.status #might come in handy in the future
    
    def getPriority(self):
        return self.priority
    
    def getTextPriority(self):
        ref_table = {1:"Normal", 2:'Medium',3:"High"}
        try:
            return  ref_table[self.priority]
        except:
            return "Not Defined"
    
    def getTextStatus(self):
        ref_table = {0:"New", 1:'In Progress',2:"Completed",3:"Cancelled",4:"Error"}
        try:
            return  ref_table[self.status]
        except:
            return "Not Defined"
        
    def updateDeadline(self,new_deadline):
        self.deadline=new_deadline
    
    def updatePriority(self,new_priority):
        self.priority = new_priority
    
    def updateStatus(self,new_status):
        self.status = new_status
    #I didnt add an update recipes because it would add complexity in the future if someone can alter the recipes
    
    def recipeInProgress(self,recipe):
        print(recipe)
        print(self.r_pending)
        self.r_pending.remove(recipe)
        self.r_inProgress.append(recipe)
        self.status = 1
    
    def recipeCompleted(self, recipe):
        self.r_completed.append(recipe)
        self.r_inProgress.remove(recipe)
        if self.r_pending == [] and self.r_inProgress == []:
            self.status = 2
            
    def __str__(self):
        return f'Job ID: {self.jobId} \nRecipes: {self.recipes} \nQuantity: {self.quantity} \nDeadline: {self.deadline} \nPriority: {self.getTextPriority()} \nStatus: {self.getTextStatus()} \nIn Progress: {self.r_inProgress} \nCompleted: {100*len(self.r_completed)/len(self.recipes)} %'

## Alternative Environment

In [19]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
import matplotlib.pyplot as plt
import pandas as pd
import datetime

#Base source: https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html


class FactoryEnv(gym.Env):
    """Custom Environment that follows gym interface."""

    metadata = {"render_modes": ["human"]}

    def __init__(self, data_path):
        super().__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        self.jobs = []
        self.machines = []
        self.tasks=[] #these will be my equivalent to the jobs in Tassel approach
        self.start_timestamp = datetime.datetime.now().timestamp()
        self.pickUpJobs(data_path)
        self.pickUpMachines(data_path)
        
        self.expandJobsToTasks()
                
        self.action_space = spaces.Discrete(len(self.machines)*len(self.tasks))
        self.observation_space = spaces.Box(low=0, high=1, shape=(len(self.jobs), len(self.machines)), dtype=int)
        
    def pickUpJobs(self,path):
        jobs_df = pd.read_csv(path+"jobs.csv")
        for i,r in jobs_df.iterrows():
            r_rec = r["Recipes"].replace("'","")
            r_rec = r_rec.split(',')
            j = Job(jobId=r["Id"], recipes=r_rec, quantity=r["Quantity"],deadline=r["Deadline"], priority=r["Priority"])
            self.jobs.append(j)
            
    def pickUpMachines(self,path):
        machines_df = pd.read_csv(path+"machines.csv")
        for i,m in machines_df.iterrows():
            m_rec = m["Recipes"].replace("'","")
            m_rec = m_rec.split(',')
            m = Machine(m_type=m["Type"], k_recipes=m_rec, cap=m["Capacity"])
            self.machines.append(m)
    
    def expandJobsToTasks(self):
        for j in self.jobs:
            for r in j.getRecipes():
                t = (r, j.getQuantity(),j.getJobId(),j.getDeadline(),j.getPriority()) # task = Tuple(Recipe, Quantity, Parent_Job, Deadline, Priority)
                self.tasks.append(t)

    def render(self):
        print("/********MACHINES********/")
        for m in self.machines:
            print(m)
            print("/-------------------------/")
        print("/**********JOBS**********/")
        for j in self.jobs:
            print(j)
            print("/-------------------------/")
        print("/**********TASKS**********/")
        print(self.tasks)
        return (self.jobs,self.machines)

In [20]:
env = FactoryEnv(data_path ='./')

In [21]:
env.render()

/********MACHINES********/
Type: A 
Known Recipes: ['A1', 'A2'] 
Tray Capacity: 10000 
Status: Free
Working on recipe:  for the following Jobs: 
/-------------------------/
Type: A 
Known Recipes: ['A2'] 
Tray Capacity: 10000 
Status: Free
Working on recipe:  for the following Jobs: 
/-------------------------/
/**********JOBS**********/
Job ID: ID-0000 
Recipes: ['A1', 'A2'] 
Quantity: 3 
Deadline: 24-04-01 
Priority: Medium 
Status: New 
In Progress: [] 
Completed: 0.0 %
/-------------------------/
Job ID: ID-0001 
Recipes: ['A2'] 
Quantity: 10 
Deadline: 24-04-14 
Priority: Normal 
Status: New 
In Progress: [] 
Completed: 0.0 %
/-------------------------/
Job ID: ID-0002 
Recipes: ['A1', 'A2'] 
Quantity: 5 
Deadline: 24-04-02 
Priority: High 
Status: New 
In Progress: [] 
Completed: 0.0 %
/-------------------------/
/**********TASKS**********/
[('A1', 3, 'ID-0000', '24-04-01', 2), ('A2', 3, 'ID-0000', '24-04-01', 2), ('A2', 10, 'ID-0001', '24-04-14', 1), ('A1', 5, 'ID-0002', '24-04-

([<__main__.Job at 0x21c75d60110>,
 [<__main__.Machine at 0x21c75d61010>, <__main__.Machine at 0x21c75d61cd0>])

## Tassel Environment:

In [34]:
import bisect
import datetime
import random

import pandas as pd
# import gym
import numpy as np
import plotly.figure_factory as ff
from pathlib import Path


class JssEnv(gym.Env):
    def __init__(self, env_config=None):
        """
        This environment model the job shop scheduling problem as a single agent problem:

        -The actions correspond to a job allocation + one action for no allocation at this time step (NOPE action)

        -We keep a time with next possible time steps

        -Each time we allocate a job, the end of the job is added to the stack of time steps

        -If we don't have a legal action (i.e. we can't allocate a job),
        we automatically go to the next time step until we have a legal action

        -
        :param env_config: Ray dictionary of config parameter
        """
        if env_config is None:
            env_config = {
                "instance_path": "ta80.txt"
            }
        instance_path = env_config["instance_path"]

        # initial values for variables used for instance
        self.jobs = 0
        self.machines = 0
        self.instance_matrix = None
        self.jobs_length = None
        self.max_time_op = 0
        self.max_time_jobs = 0
        self.nb_legal_actions = 0
        self.nb_machine_legal = 0
        # initial values for variables used for solving (to reinitialize when reset() is called)
        self.solution = None
        self.last_solution = None
        self.last_time_step = float("inf")
        self.current_time_step = float("inf")
        self.next_time_step = list()
        self.next_jobs = list()
        self.legal_actions = None
        self.time_until_available_machine = None
        self.time_until_finish_current_op_jobs = None
        self.todo_time_step_job = None
        self.total_perform_op_time_jobs = None
        self.needed_machine_jobs = None
        self.total_idle_time_jobs = None
        self.idle_time_jobs_last_op = None
        self.state = None
        self.illegal_actions = None
        self.action_illegal_no_op = None
        self.machine_legal = None
        # initial values for variables used for representation
        self.start_timestamp = datetime.datetime.now().timestamp()
        self.sum_op = 0
        with open(instance_path, "r") as instance_file:
            for line_cnt, line_str in enumerate(instance_file, start=1):
                split_data = list(map(int, line_str.split()))

                if line_cnt == 1:
                    self.jobs, self.machines = split_data
                    self.instance_matrix = np.zeros((self.jobs, self.machines), dtype=(int, 2))
                    self.jobs_length = np.zeros(self.jobs, dtype=int)
                else:
                    assert len(split_data) % 2 == 0 and len(split_data) // 2 == self.machines
                    job_nb = line_cnt - 2
                    for i in range(0, len(split_data), 2):
                        machine, time = split_data[i], split_data[i + 1]
                        self.instance_matrix[job_nb][i // 2] = (machine, time)
                        self.max_time_op = max(self.max_time_op, time)
                        self.jobs_length[job_nb] += time
                        self.sum_op += time
        self.max_time_jobs = max(self.jobs_length)
        print("jobs: ")
        print(self.jobs)
        print("machines: ")
        print(self.machines)
        print("Insrance matrix: ")
        print(self.instance_matrix)
        print("jobs_length: ")
        print(self.jobs_length)
        print("sum op: ")
        print(self.sum_op)
        print("max_time_jobs: ")
        print(self.max_time_jobs)
        # check the parsed data are correct
        assert self.max_time_op > 0
        assert self.max_time_jobs > 0
        assert self.jobs > 0
        assert self.machines > 1, "We need at least 2 machines"
        assert self.instance_matrix is not None
        # allocate a job + one to wait
        self.action_space = gym.spaces.Discrete(self.jobs + 1)
        # used for plotting
        self.colors = [
            tuple([random.random() for _ in range(3)]) for _ in range(self.machines)
        ]
        """
        matrix with the following attributes for each job:
            -Legal job
            -Left over time on the current op
            -Current operation %
            -Total left over time
            -When next machine available
            -Time since IDLE: 0 if not available, time otherwise
            -Total IDLE time in the schedule
        """
        self.observation_space = gym.spaces.Dict(
            {
                "action_mask": gym.spaces.Box(0, 1, shape=(self.jobs + 1,)),
                "real_obs": gym.spaces.Box(
                    low=0.0, high=1.0, shape=(self.jobs, 7), dtype=float
                ),
            }
        )

    def _get_current_state_representation(self):
        self.state[:, 0] = self.legal_actions[:-1]
        return {
            "real_obs": self.state,
            "action_mask": self.legal_actions,
        }

    def get_legal_actions(self):
        return self.legal_actions

    def reset(self):
        self.current_time_step = 0
        self.next_time_step = list()
        self.next_jobs = list()
        self.nb_legal_actions = self.jobs
        self.nb_machine_legal = 0
        # represent all the legal actions
        self.legal_actions = np.ones(self.jobs + 1, dtype=bool)
        self.legal_actions[self.jobs] = False
        # used to represent the solution
        self.solution = np.full((self.jobs, self.machines), -1, dtype=int)
        self.time_until_available_machine = np.zeros(self.machines, dtype=int)
        self.time_until_finish_current_op_jobs = np.zeros(self.jobs, dtype=int)
        self.todo_time_step_job = np.zeros(self.jobs, dtype=int)
        self.total_perform_op_time_jobs = np.zeros(self.jobs, dtype=int)
        self.needed_machine_jobs = np.zeros(self.jobs, dtype=int)
        self.total_idle_time_jobs = np.zeros(self.jobs, dtype=int)
        self.idle_time_jobs_last_op = np.zeros(self.jobs, dtype=int)
        self.illegal_actions = np.zeros((self.machines, self.jobs), dtype=bool)
        self.action_illegal_no_op = np.zeros(self.jobs, dtype=bool)
        self.machine_legal = np.zeros(self.machines, dtype=bool)
        for job in range(self.jobs):
            needed_machine = self.instance_matrix[job][0][0]
            self.needed_machine_jobs[job] = needed_machine
            if not self.machine_legal[needed_machine]:
                self.machine_legal[needed_machine] = True
                self.nb_machine_legal += 1
        self.state = np.zeros((self.jobs, 7), dtype=float)
        return self._get_current_state_representation()

    def _prioritization_non_final(self):
        if self.nb_machine_legal >= 1:
            for machine in range(self.machines):
                if self.machine_legal[machine]:
                    final_job = list()
                    non_final_job = list()
                    min_non_final = float("inf")
                    for job in range(self.jobs):
                        if (
                            self.needed_machine_jobs[job] == machine
                            and self.legal_actions[job]
                        ):
                            if self.todo_time_step_job[job] == (self.machines - 1):
                                final_job.append(job)
                            else:
                                current_time_step_non_final = self.todo_time_step_job[
                                    job
                                ]
                                time_needed_legal = self.instance_matrix[job][
                                    current_time_step_non_final
                                ][1]
                                machine_needed_nextstep = self.instance_matrix[job][
                                    current_time_step_non_final + 1
                                ][0]
                                if (
                                    self.time_until_available_machine[
                                        machine_needed_nextstep
                                    ]
                                    == 0
                                ):
                                    min_non_final = min(
                                        min_non_final, time_needed_legal
                                    )
                                    non_final_job.append(job)
                    if len(non_final_job) > 0:
                        for job in final_job:
                            current_time_step_final = self.todo_time_step_job[job]
                            time_needed_legal = self.instance_matrix[job][
                                current_time_step_final
                            ][1]
                            if time_needed_legal > min_non_final:
                                self.legal_actions[job] = False
                                self.nb_legal_actions -= 1

    def _check_no_op(self):
        self.legal_actions[self.jobs] = False
        if (
            len(self.next_time_step) > 0
            and self.nb_machine_legal <= 3
            and self.nb_legal_actions <= 4
        ):
            machine_next = set()
            next_time_step = self.next_time_step[0]
            max_horizon = self.current_time_step
            max_horizon_machine = [
                self.current_time_step + self.max_time_op for _ in range(self.machines)
            ]
            for job in range(self.jobs):
                if self.legal_actions[job]:
                    time_step = self.todo_time_step_job[job]
                    machine_needed = self.instance_matrix[job][time_step][0]
                    time_needed = self.instance_matrix[job][time_step][1]
                    end_job = self.current_time_step + time_needed
                    if end_job < next_time_step:
                        return
                    max_horizon_machine[machine_needed] = min(
                        max_horizon_machine[machine_needed], end_job
                    )
                    max_horizon = max(max_horizon, max_horizon_machine[machine_needed])
            for job in range(self.jobs):
                if not self.legal_actions[job]:
                    if (
                        self.time_until_finish_current_op_jobs[job] > 0
                        and self.todo_time_step_job[job] + 1 < self.machines
                    ):
                        time_step = self.todo_time_step_job[job] + 1
                        time_needed = (
                            self.current_time_step
                            + self.time_until_finish_current_op_jobs[job]
                        )
                        while (
                            time_step < self.machines - 1 and max_horizon > time_needed
                        ):
                            machine_needed = self.instance_matrix[job][time_step][0]
                            if (
                                max_horizon_machine[machine_needed] > time_needed
                                and self.machine_legal[machine_needed]
                            ):
                                machine_next.add(machine_needed)
                                if len(machine_next) == self.nb_machine_legal:
                                    self.legal_actions[self.jobs] = True
                                    return
                            time_needed += self.instance_matrix[job][time_step][1]
                            time_step += 1
                    elif (
                        not self.action_illegal_no_op[job]
                        and self.todo_time_step_job[job] < self.machines
                    ):
                        time_step = self.todo_time_step_job[job]
                        machine_needed = self.instance_matrix[job][time_step][0]
                        time_needed = (
                            self.current_time_step
                            + self.time_until_available_machine[machine_needed]
                        )
                        while (
                            time_step < self.machines - 1 and max_horizon > time_needed
                        ):
                            machine_needed = self.instance_matrix[job][time_step][0]
                            if (
                                max_horizon_machine[machine_needed] > time_needed
                                and self.machine_legal[machine_needed]
                            ):
                                machine_next.add(machine_needed)
                                if len(machine_next) == self.nb_machine_legal:
                                    self.legal_actions[self.jobs] = True
                                    return
                            time_needed += self.instance_matrix[job][time_step][1]
                            time_step += 1

    def step(self, action: int):
        reward = 0.0
        if action == self.jobs:
            self.nb_machine_legal = 0
            self.nb_legal_actions = 0
            for job in range(self.jobs):
                if self.legal_actions[job]:
                    self.legal_actions[job] = False
                    needed_machine = self.needed_machine_jobs[job]
                    self.machine_legal[needed_machine] = False
                    self.illegal_actions[needed_machine][job] = True
                    self.action_illegal_no_op[job] = True
            while self.nb_machine_legal == 0:
                reward -= self.increase_time_step()
            scaled_reward = self._reward_scaler(reward)
            self._prioritization_non_final()
            self._check_no_op()
            return (
                self._get_current_state_representation(),
                scaled_reward,
                self._is_done(),
                {},
            )
        else:
            current_time_step_job = self.todo_time_step_job[action]
            machine_needed = self.needed_machine_jobs[action]
            time_needed = self.instance_matrix[action][current_time_step_job][1]
            reward += time_needed
            self.time_until_available_machine[machine_needed] = time_needed
            self.time_until_finish_current_op_jobs[action] = time_needed
            self.state[action][1] = time_needed / self.max_time_op
            to_add_time_step = self.current_time_step + time_needed
            if to_add_time_step not in self.next_time_step:
                index = bisect.bisect_left(self.next_time_step, to_add_time_step)
                self.next_time_step.insert(index, to_add_time_step)
                self.next_jobs.insert(index, action)
            self.solution[action][current_time_step_job] = self.current_time_step
            for job in range(self.jobs):
                if (
                    self.needed_machine_jobs[job] == machine_needed
                    and self.legal_actions[job]
                ):
                    self.legal_actions[job] = False
                    self.nb_legal_actions -= 1
            self.nb_machine_legal -= 1
            self.machine_legal[machine_needed] = False
            for job in range(self.jobs):
                if self.illegal_actions[machine_needed][job]:
                    self.action_illegal_no_op[job] = False
                    self.illegal_actions[machine_needed][job] = False
            # if we can't allocate new job in the current timestep, we pass to the next one
            while self.nb_machine_legal == 0 and len(self.next_time_step) > 0:
                reward -= self.increase_time_step()
            self._prioritization_non_final()
            self._check_no_op()
            # we then need to scale the reward
            scaled_reward = self._reward_scaler(reward)
            return (
                self._get_current_state_representation(),
                scaled_reward,
                self._is_done(),
                {},
            )

    def _reward_scaler(self, reward):
        return reward / self.max_time_op

    def increase_time_step(self):
        """
        The heart of the logic his here, we need to increase every counter when we have a nope action called
        and return the time elapsed
        :return: time elapsed
        """
        hole_planning = 0
        next_time_step_to_pick = self.next_time_step.pop(0)
        self.next_jobs.pop(0)
        difference = next_time_step_to_pick - self.current_time_step
        self.current_time_step = next_time_step_to_pick
        for job in range(self.jobs):
            was_left_time = self.time_until_finish_current_op_jobs[job]
            if was_left_time > 0:
                performed_op_job = min(difference, was_left_time)
                self.time_until_finish_current_op_jobs[job] = max(
                    0, self.time_until_finish_current_op_jobs[job] - difference
                )
                self.state[job][1] = (
                    self.time_until_finish_current_op_jobs[job] / self.max_time_op
                )
                self.total_perform_op_time_jobs[job] += performed_op_job
                self.state[job][3] = (
                    self.total_perform_op_time_jobs[job] / self.max_time_jobs
                )
                if self.time_until_finish_current_op_jobs[job] == 0:
                    self.total_idle_time_jobs[job] += difference - was_left_time
                    self.state[job][6] = self.total_idle_time_jobs[job] / self.sum_op
                    self.idle_time_jobs_last_op[job] = difference - was_left_time
                    self.state[job][5] = self.idle_time_jobs_last_op[job] / self.sum_op
                    self.todo_time_step_job[job] += 1
                    self.state[job][2] = self.todo_time_step_job[job] / self.machines
                    if self.todo_time_step_job[job] < self.machines:
                        self.needed_machine_jobs[job] = self.instance_matrix[job][
                            self.todo_time_step_job[job]
                        ][0]
                        self.state[job][4] = (
                            max(
                                0,
                                self.time_until_available_machine[
                                    self.needed_machine_jobs[job]
                                ]
                                - difference,
                            )
                            / self.max_time_op
                        )
                    else:
                        self.needed_machine_jobs[job] = -1
                        # this allow to have 1 is job is over (not 0 because, 0 strongly indicate that the job is a
                        # good candidate)
                        self.state[job][4] = 1.0
                        if self.legal_actions[job]:
                            self.legal_actions[job] = False
                            self.nb_legal_actions -= 1
            elif self.todo_time_step_job[job] < self.machines:
                self.total_idle_time_jobs[job] += difference
                self.idle_time_jobs_last_op[job] += difference
                self.state[job][5] = self.idle_time_jobs_last_op[job] / self.sum_op
                self.state[job][6] = self.total_idle_time_jobs[job] / self.sum_op
        for machine in range(self.machines):
            if self.time_until_available_machine[machine] < difference:
                empty = difference - self.time_until_available_machine[machine]
                hole_planning += empty
            self.time_until_available_machine[machine] = max(
                0, self.time_until_available_machine[machine] - difference
            )
            if self.time_until_available_machine[machine] == 0:
                for job in range(self.jobs):
                    if (
                        self.needed_machine_jobs[job] == machine
                        and not self.legal_actions[job]
                        and not self.illegal_actions[machine][job]
                    ):
                        self.legal_actions[job] = True
                        self.nb_legal_actions += 1
                        if not self.machine_legal[machine]:
                            self.machine_legal[machine] = True
                            self.nb_machine_legal += 1
        return hole_planning

    def _is_done(self):
        if self.nb_legal_actions == 0:
            self.last_time_step = self.current_time_step
            self.last_solution = self.solution
            return True
        return False

    def render(self, mode="human"):
        df = []
        for job in range(self.jobs):
            i = 0
            while i < self.machines and self.solution[job][i] != -1:
                dict_op = dict()
                dict_op["Task"] = "Job {}".format(job)
                start_sec = self.start_timestamp + self.solution[job][i]
                finish_sec = start_sec + self.instance_matrix[job][i][1]
                dict_op["Start"] = datetime.datetime.fromtimestamp(start_sec)
                dict_op["Finish"] = datetime.datetime.fromtimestamp(finish_sec)
                dict_op["Resource"] = "Machine {}".format(
                    self.instance_matrix[job][i][0]
                )
                df.append(dict_op)
                i += 1
        fig = None
        if len(df) > 0:
            df = pd.DataFrame(df)
            fig = ff.create_gantt(
                df,
                index_col="Resource",
                colors=self.colors,
                show_colorbar=True,
                group_tasks=True,
            )
            fig.update_yaxes(
                autorange="reversed"
            )  # otherwise tasks are listed from the bottom up
        return fig


# if __name__ == '__main__':
#     env = JssEnv()
#     obs = env.reset()
#     done = False
#     cum_reward = 0
#     while not done:
#         legal_actions = obs["action_mask"]
#         actions = np.random.choice(
#             len(legal_actions), 1, p=(legal_actions / legal_actions.sum())
#         )[0]
#         obs, rewards, done, _ = env.step(actions)
#         cum_reward += rewards
#     print(f"Cumulative reward: {cum_reward}")


In [35]:
envT = JssEnv()

jobs: 
100
machines: 
20
Insrance matrix: 
[[[ 9 54]
  [12 87]
  [ 4 61]
  ...
  [ 2 32]
  [19 96]
  [ 6 62]]

 [[17 68]
  [13 15]
  [19 22]
  ...
  [ 4 14]
  [ 8 22]
  [12 66]]

 [[17 68]
  [10 70]
  [ 3 84]
  ...
  [ 1 56]
  [ 9 88]
  [13 71]]

 ...

 [[ 6  4]
  [10 55]
  [ 4 31]
  ...
  [16 12]
  [ 5  2]
  [18 61]]

 [[11 70]
  [ 9 69]
  [ 8 26]
  ...
  [ 6  2]
  [ 1 53]
  [15 28]]

 [[11 39]
  [12 56]
  [ 9 87]
  ...
  [ 6 30]
  [10 46]
  [ 3 21]]]
jobs_length: 
[1048  968 1078  910 1072  955  833  909 1121 1007 1206 1175  927  865
  983 1124  908  808  911 1075 1115 1009 1040  884  872  870  988  869
 1022  968  906  807  972  941 1184 1219  845 1036  813  869 1064 1091
  810 1113 1027 1091  813  911  800  880  851 1031 1004 1092  997 1180
  966 1180 1226  817 1045  931 1019  797  911  794  903  856 1055  922
 1145  977 1098  929  739  797  969  945 1159 1028  833  934  951  824
  991  927  852  869  976 1108  813 1102 1207  832  803  877  882  782
 1087  972]
sum op: 
96697
max_t